## Récupération des articles

In [3]:
from bs4 import BeautifulSoup
import requests

In [4]:
def scraping_ecofin(champ):

  liens = []

  for debut in range(0, champ+10, 10):

    url = f"https://www.agenceecofin.com/a-la-une/recherche-article/articles?submit_x=33&submit_y=16&filterTousLesFils=Tous&filterCategories=Sous-rubrique&filterDateFrom=2025-02-24&filterDateTo=2025-03-31&filterFrench=French&userSearch=0&testlimitstart={debut}"

    page = requests.get(url)

    soup = BeautifulSoup(page.text, 'html')

    tables = soup.findAll('table', class_='ts')
    for table in tables:
      a_tags = table.findAll('a')
      for a_tag in a_tags:
        href = a_tag.get("href")
        if href:
            href = "https://www.agenceecofin.com"+ href
            if href not in liens:
              liens.append(href)
  return liens

In [5]:
liens = scraping_ecofin(1050)

<ipython-input-4-5f2b7526be1f>:13: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  tables = soup.findAll('table', class_='ts')
<ipython-input-4-5f2b7526be1f>:15: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  a_tags = table.findAll('a')


In [6]:
liens

['https://www.agenceecofin.com/actualites/3103-127141-la-smb-deploie-ses-10-ans-dexperience-lors-de-la-fastmarkets-bauxite-alumina-conference-2025',
 'https://www.agenceecofin.com/actualites-agro/3103-127140-nigeria-un-pret-de-80-millions-du-japon-pour-faciliter-l-acces-aux-intrants-agricoles',
 'https://www.agenceecofin.com/actualites/3103-127139-le-kenya-hote-de-la-semaine-mondiale-des-relations-publiques-africaines-2025',
 'https://www.agenceecofin.com/actualites-industries/3103-127138-cote-d-ivoire-le-debut-des-travaux-eies-du-projet-aurifere-boundiali-prevu-pour-avril',
 'https://www.agenceecofin.com/actualites-industries/3103-127137-cmoc-etablit-des-resultats-record-en-2024-et-entre-dans-le-top-10-mondial-des-producteurs-de-cuivre',
 'https://www.agenceecofin.com/actualites/3103-127136-l-ue-et-le-nigeria-s-orientent-vers-la-creation-d-un-cadre-formel-de-dialogue-economique',
 'https://www.agenceecofin.com/actualites-infrastructures/3103-127135-le-ghana-va-consacrer-180-millions-a

In [7]:
import re
def scraping_pages(url):

  texte = []

  page = requests.get(url)

  soup = BeautifulSoup(page.text, 'html')

  divs = soup.findAll("div", class_="itemIntroText")
  for div in divs:
    a_tags = div.findAll("p", class_="texte textearticle")
    for a_tag in a_tags:
      text_content = a_tag.text
      texte.append(text_content)
  return texte

In [8]:
contenu = []
for lien in liens:
  contenu.append(scraping_pages(lien))

<ipython-input-7-581277b2f2ee>:10: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  divs = soup.findAll("div", class_="itemIntroText")
<ipython-input-7-581277b2f2ee>:12: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  a_tags = div.findAll("p", class_="texte textearticle")


In [9]:
import pandas as pd

ta = {'Lien':liens, 'Texte':contenu}
df = pd.DataFrame(ta)

df.to_csv('my_data.csv', index=False)

In [10]:
df.head()

,Lien,Texte
0,https://www.agenceecofin.com/actualites/3103-1...,"[(SMB) - Forte de 10 ans d’existence, la Socié..."
1,https://www.agenceecofin.com/actualites-agro/3...,"[(Agence Ecofin) - Au Nigeria, le secteur agri..."
2,https://www.agenceecofin.com/actualites/3103-1...,"[(Agence Ecofin) - Chaque année, les professio..."
3,https://www.agenceecofin.com/actualites-indust...,[(Agence Ecofin) - Aurum a publié en janvier u...
4,https://www.agenceecofin.com/actualites-indust...,[(CMOC) - CMOC Group Limited (CMOC) a annoncé ...


## Nettoyage des éléments de la colonne texte.

In [22]:
df['Texte'] = df['Texte'].apply(lambda x: " ".join([re.sub(r'[\[\]\,\(\)\"\\\-\'\xa0]', ' ', item) for item in x]) if isinstance(x, list) else re.sub(r'[\[\]\,\(\)\"\\\-\'\xa0]', ' ', x))